In [1]:
!pip install -U bitsandbytes transformers accelerate peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [11]:
import os
os.environ['HF_TOKEN'] = ''

In [5]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
df_qa = getDF('qa_Beauty.json.gz')

In [6]:
from sklearn.model_selection import train_test_split

# Determine the minimum count of the two question types
#min_count = df_qa['questionType'].value_counts().min()

# Sample an equal number of rows for each question type
df_open_ended = df_qa[df_qa['questionType'] == 'open-ended'].sample(n=15000, random_state=42)
df_yes_no = df_qa[df_qa['questionType'] == 'yes/no'].sample(n=15000, random_state=42)

# Concatenate the two dataframes to create the balanced sample
df_sample = pd.concat([df_open_ended, df_yes_no])

# Shuffle the dataframe to mix the question types
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)

display(df_sample['questionType'].value_counts())

,count
questionType,
open-ended,15000
yes/no,15000


In [7]:
from datasets import Dataset

training_data = []
for _, row in df_sample.iterrows():
    prompt = row['question']  # only question
    answer = row['answer']    # only answer
    training_data.append({'prompt': prompt, 'answer': answer})

dataset = Dataset.from_list(training_data)

# Split into train/val/test
dataset_split = dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = dataset_split['train']

test_valid_split = dataset_split['test'].train_test_split(test_size=0.5, seed=42)

val_dataset = test_valid_split['train']
test_dataset = test_valid_split['test']


In [8]:
from transformers import AutoTokenizer
checkpoint_path = "/content/drive/MyDrive/llama1b_instruct_control/checkpoint-1300"
#model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token  # required

def tokenize_with_labels(example):
    """
    Tokenize and create labels where:
    - Prompt tokens have label = -100 (ignored in loss)
    - Answer tokens have label = token_id (used in loss)
    """
    # Format the conversation
    messages = [
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["answer"]}
    ]

    # Get the full formatted text
    full_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    # Tokenize the full conversation
    full_tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=128,
        padding=False
    )

    # Now tokenize just the prompt part to find where answer starts
    prompt_messages = [
        {"role": "user", "content": example["prompt"]},
    ]
    prompt_text = tokenizer.apply_chat_template(
        prompt_messages,
        tokenize=False,
        add_generation_prompt=True  # This adds the assistant header
    )

    prompt_tokenized = tokenizer(
        prompt_text,
        truncation=True,
        max_length=128,
        padding=False
    )

    # Create labels: -100 for prompt, actual token_ids for answer
    prompt_length = len(prompt_tokenized["input_ids"])
    labels = [-100] * prompt_length + full_tokenized["input_ids"][prompt_length:]

    # Ensure labels and input_ids have the same length
    labels = labels[:len(full_tokenized["input_ids"])]

    return {
        "input_ids": full_tokenized["input_ids"],
        "attention_mask": full_tokenized["attention_mask"],
        "labels": labels
    }

print("Tokenizing train dataset...")
train_dataset_tokenized = train_dataset.map(
    tokenize_with_labels,
    batched=False,  # Changed to False
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train data"
)

print("Tokenizing validation dataset...")
val_dataset_tokenized = val_dataset.map(
    tokenize_with_labels,
    batched=False,  # Changed to False
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation data"
)

print("Tokenizing test dataset...")
test_dataset_tokenized = test_dataset.map(
    tokenize_with_labels,
    batched=False,  # Changed to False
    remove_columns=test_dataset.column_names,
    desc="Tokenizing test data"
)

Tokenizing train dataset...


Tokenizing train data:   0%|          | 0/21000 [00:00<?, ? examples/s]

Tokenizing validation dataset...


Tokenizing validation data:   0%|          | 0/4500 [00:00<?, ? examples/s]

Tokenizing test dataset...


Tokenizing test data:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [3]:
import wandb

# Log in to your W&B account
# You will be prompted to enter your API key
wandb.login()
wandb.init(project="new2")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: e1083039 (e1083039-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title
import zipfile

zip_file_path = '/content/drive/MyDrive/checkpoint-700.zip'
extract_dir = '/content' # This should match the directory used in Kx4palri9BUS

# Create the directory if it doesn't exist
import os
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'Files extracted to: {extract_dir}')

Files extracted to: /content


In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

In [15]:
# @title
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

#model_name = "meta-llama/Llama-3.2-1B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.2,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    init_lora_weights=True
)

model = get_peft_model(model, peft_config)
model.enable_input_require_grads()

In [10]:




# -------------------------------
# Model + QLoRA + LoRA setup
# -------------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load pretrained checkpoint (4-bit + LoRA)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

# Prepare model for k-bit training (gradient checkpointing, etc.)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# LoRA configuration (should match previous)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.2,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    init_lora_weights=True
)

model = get_peft_model(model, peft_config)
model.enable_input_require_grads()

# -------------------------------
# Data collator
# -------------------------------
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

# -------------------------------
# Training arguments
# -------------------------------
training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/llama1b_instruct_control",
    num_train_epochs=1.5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    warmup_ratio=0.05,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    bf16=True,
    fp16=False,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="wandb",
    remove_unused_columns=False,
    max_grad_norm=0.5,
    dataloader_num_workers=2,
    weight_decay=0.01,
    group_by_length=True,
    gradient_checkpointing=True,
    dataloader_pin_memory=True,
)

# -------------------------------
# Custom Trainer for logging
# -------------------------------
class MetricsTracker(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_losses = []
        self.eval_losses = []
        self.learning_rates = []
        self.steps = []

    def log(self, logs, start_time=None):
        super().log(logs, start_time=start_time)
        if "loss" in logs:
            self.train_losses.append(logs["loss"])
            self.steps.append(self.state.global_step)
        if "learning_rate" in logs:
            self.learning_rates.append(logs["learning_rate"])
        if "eval_loss" in logs:
            self.eval_losses.append(logs["eval_loss"])

# -------------------------------
# Trainer
# -------------------------------
trainer = MetricsTracker(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2,
                                     early_stopping_threshold=0.01)]
)

# -------------------------------
# Resume training from checkpoint
# -------------------------------



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,3.460100,3.513046
200,3.281300,3.356209
300,3.204500,3.267010
400,3.180200,3.232249
500,3.139200,3.232746
600,3.079900,3.210042
700,3.120300,3.181468
800,3.116400,3.145865
900,3.071000,3.133326
1000,3.019200,3.113068


TrainOutput(global_step=1300, training_loss=3.1837025216909556, metrics={'train_runtime': 11976.0089, 'train_samples_per_second': 2.63, 'train_steps_per_second': 0.164, 'total_flos': 1.0672165462474752e+16, 'train_loss': 3.1837025216909556, 'epoch': 0.9904761904761905})

In [12]:
trainer.train(resume_from_checkpoint=checkpoint_path)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
1400,3.070100,3.096973
1500,2.942100,3.094493


TrainOutput(global_step=1500, training_loss=0.3977840881347656, metrics={'train_runtime': 1949.9617, 'train_samples_per_second': 16.154, 'train_steps_per_second': 1.01, 'total_flos': 1.2362277809553408e+16, 'train_loss': 0.3977840881347656, 'epoch': 1.1424761904761904})

evaluation

In [13]:
# @title
eval_results = trainer.evaluate(test_dataset_tokenized)
import math
eval_results["perplexity"] = math.exp(eval_results["eval_loss"])

print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 3.084606885910034, 'eval_runtime': 462.8327, 'eval_samples_per_second': 9.723, 'eval_steps_per_second': 1.216, 'epoch': 1.1424761904761904, 'perplexity': 21.858872122211203}


In [14]:
trainer.save_model("/content/drive/MyDrive/llama1b_instruct/model")